In [1]:
using PyCall, PyPlot, JuMP
using ClimateMARGO

In [2]:
] add Combinatorics

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %                                 ]  15.2 % %                     ]  45.4 %2 %           ]  70.4 %Fetching: [==============================>          ]  73.4 %    ]  88.5 %  Updating registry at `~/.julia/registries/MimiRegistry`
  Updating git-repo `https://github.com/mimiframework/MimiRegistry.git`
 Resolving package versions...
   Cloning [53c48c17-4a7d-5ca2-90c5-79b7896eea93] FixedPointNumbers from https://github.com/JuliaMath/FixedPointNumbers.jl.git
 Installed FixedPointNumbers ─ v0.8.1========>]  100.0 %.0 % [========>                                ]  19.6 %]  39.1 %========================>                ]  58.8 %]  78.2 %Fetching: [=================================>       ]  81.2 %
  Updating git-repo `https://github.com/JuliaGraphics/ColorTypes.jl.git`
 Installed ColorTypes ──────── v0.10.4

In [3]:
using Combinatorics

# Optimizating the Global Climate Action Policy Portfolio

In [4]:
model_og = ClimateModel();
max_deployment_og = Dict("mitigate"=>1., "remove"=>1., "geoeng"=>1., "adapt"=>0.4)

Dict{String,Float64} with 4 entries:
  "geoeng"   => 1.0
  "mitigate" => 1.0
  "remove"   => 1.0
  "adapt"    => 0.4

In [5]:
controls = collect(keys(max_deployment_og));

In [6]:
control_permutations = combinations(controls) |> collect

15-element Array{Array{String,1},1}:
 ["geoeng"]                               
 ["mitigate"]                             
 ["remove"]                               
 ["adapt"]                                
 ["geoeng", "mitigate"]                   
 ["geoeng", "remove"]                     
 ["geoeng", "adapt"]                      
 ["mitigate", "remove"]                   
 ["mitigate", "adapt"]                    
 ["remove", "adapt"]                      
 ["geoeng", "mitigate", "remove"]         
 ["geoeng", "mitigate", "adapt"]          
 ["geoeng", "remove", "adapt"]            
 ["mitigate", "remove", "adapt"]          
 ["geoeng", "mitigate", "remove", "adapt"]

In [7]:
results = Dict()

for control_permutation in control_permutations
    model = ClimateModel();
    
    # Assert that some controls not be deployed
    max_deployment = copy(max_deployment_og)
    for control in controls
        if ~(control in control_permutation)
            max_deployment[control] = 0.
            # Override initial condition constraints
            if ~isnothing(getfield(model.economics, Symbol(string(control,"_init"))))
                setfield!(model.economics, Symbol(string(control,"_init")), 0.)
            end
        end
    end
    @time m = optimize_controls!(model, max_deployment = max_deployment, temp_goal=2.0);
    
    results[control_permutation] = Dict("model" => model, "NPV" => discounted_total_control_cost(model), "status" => raw_status(m))
    
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Solve_Succeeded
 37.607116 seconds (114.87 M allocations: 6.511 GiB, 6.04% gc time)
Solve_Succeeded
  0.273680 seconds (368.39 k allocations: 8.579 MiB)
Infeasible_Problem_Detected
  0.827848 seconds (1.66 M allocations: 29.776 MiB, 3.53% gc time)
Infeasible_Problem_Detected
  2.330643 seconds (1.93 M allocations: 34.447 MiB)
Solve_Succeeded
  0.070496 seconds (96.33 k allocations: 4.154 MiB)
Solve_Succeeded
  0.049572 seconds (92.68 k allocations: 4.065 MiB)
Solve_Succeeded
  0.068749 seconds (103.00 k allocations: 4.259 MiB)
Solve_Succeeded
  0.077293 seconds (116.36 k allocations: 4.523 MiB)
Solve_Succ

In [8]:
min_value = Inf
for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded") & (value["NPV"] < min_value)
        min_value = value["NPV"]
    end
end

for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded")
        abs_value = Int64(round((value["NPV"])))
        rel_value = Int64(round((value["NPV"]/min_value - 1.)*100))
        print("$index, $abs_value, $rel_value%\n")
    else
        print("$index has no solution.\n")
    end
end


["geoeng", "mitigate", "remove"], 181, 8%
["remove"] has no solution.
["mitigate", "remove"], 257, 53%
["mitigate"], 322, 91%
["geoeng", "mitigate", "remove", "adapt"], 168, 0%
["geoeng", "adapt"], 304, 81%
["geoeng", "mitigate", "adapt"], 181, 8%
["mitigate", "remove", "adapt"], 236, 40%
["geoeng"], 412, 146%
["geoeng", "remove", "adapt"], 280, 67%
["mitigate", "adapt"], 277, 65%
["geoeng", "mitigate"], 199, 19%
["remove", "adapt"] has no solution.
["geoeng", "remove"], 371, 121%
["adapt"] has no solution.
